In [1]:
# %%
import numpy as np
from rotMatrix import rotationMatrixAroundY, rotationMatrixAroundZ, rotationMatrixAroundX, rotationMatrixAroundAxis, rotatePoints
#from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
from scipy.linalg import norm
#import matplotlib.pyplot as plt
import sunposition as sp
from ipywidgets import IntSlider, FloatSlider, BoundedFloatText,HBox, VBox, Layout, interactive_output
from IPython.display import display
from timeUtil import days_in_month, timeZoneToUTC
from geoElem import sunVect,parabolicCylinderSurface, planeSurface, sphereSurface, arcCircle3D, arcCircle2DAngle


#region defining functions


#endregion 

#region read-input 

# Default values
lon = 6.5  
lat = 50.45 
day = 21
month = 6
year = 2023
axRot_az=0.0
timeZone=0.0 

# Use input with default values
#lon = input(f'Enter the longitude [{lon}]: ') or lon
#lat = input(f'Enter the latitude [{lat}]: ') or lat
#day = input(f'Enter the day [{day}]: ') or day
#month = input(f'Enter the month [{month}]: ') or month
#year = input(f'Enter the year [{year}]: ') or year
#axRot_az = input(f'Enter the azimuth rotation angle [{0}]°: ') or axRot_az

timeStep=1 # [h]
#endregion
firstRun=True

def initialCompute(lon, lat, year, month, day):
    global azArray, zenArray, hourArray, endArr, startArr, ii, iiSun, iiFin
    # Initialize arrays for azimuth and zenith angles
    azArray = np.zeros(24)
    zenArray = np.zeros(24)
    hourArray = np.linspace(0, 23, 24)
    lon = float(lon)
    lat = float(lat)
    year = str(year)
    month = str(month)
    day = str(day)
    if int(month) < 10:
        month = '0' + str(month)
    if int(day) < 10:
        day = '0' + str(day)

    stringDay = year + '-' + month + '-' + day
    lon = np.array([lon])
    lat = np.array([lat])

    sp.disable_jit()
    for ii in hourArray:
        addStr = ''
        if ii < 10:
            addStr = '0'
        currTime = np.datetime64(stringDay + 'T' + addStr + str(int(ii)) + ':00:00')
        #azArray[int(ii)], zenArray[int(ii)] = sp.sunpos(currTime, lat, lon, 0)[:2]  # discard RA, dec, H
        a,b= sp.sunpos(currTime,lat,lon,0)[:2] #discard RA, dec, H 
        azArray[int(ii)]=a[0]
        zenArray[int(ii)]=b[0]

    iiSun = np.argmax((zenArray > 0) & (zenArray <= 90))
    ii = iiSun

    endArr = np.zeros((3, 24))
    startArr = np.zeros((3, 24))

    while ((zenArray[ii] <= 90) & (zenArray[ii] >= 0)):
        sunVector = np.array([np.sin(np.deg2rad(zenArray[ii])) * np.cos(np.deg2rad(azArray[ii])),
                              np.sin(np.deg2rad(zenArray[ii])) * np.sin(np.deg2rad(azArray[ii])),
                              np.cos(np.deg2rad(zenArray[ii]))])
        comp(np.deg2rad(zenArray[ii]), np.deg2rad(azArray[ii]))
        startArr[:, ii] = start[:, 0]
        endArr[:, ii] = startArr[:, ii] + sunVector * length
        ii = int(ii) + 1  # Ensure ii is an integer to avoid deprecation warnings

    iiFin = ii - 1

def comp(zenith, azimuth, axRot_az=0):
    global projSunHor, zen, az,ii, startArr, sunVector, endArr, iiSun, end2, normalVect, start, end, x, y, z, xAp, yAp, zAp, xLon, yLon, zLon, xHor, yHor, zHor, xArc, yArc, zArc, xArcZen, yArcZen, zArcZen, xArcAz, yArcAz, zArcAz, xSun, ySun, zSun, incAngle, azArray, zenArray, length,thetaPerp

    zen=np.rad2deg(zenith)  
    az=np.rad2deg(azimuth)
    length=10.
    azApparent=azimuth-np.deg2rad(axRot_az)
    x,y,z=parabolicCylinderSurface()    
    maxZ=np.max(z)
    centX=(np.max(x)+np.min(x))/2.0
    centY=(np.max(y)+np.min(y))/2.0
    start=np.array([centX,centY,maxZ]).reshape((3,1))  # Start point of the sun vector    
    xAp,yAp,zAp=planeSurface(trasl=[0,0,0])
    xLon,yLon,zLon=planeSurface(trasl=[0,0,0])
    xHor,yHor,zHor=planeSurface(trasl=[0,0,0])
    rotAngle = np.deg2rad(90)  # Rotate by 90 degrees
    rotMatr=rotationMatrixAroundY(rotAngle)
    xLon,yLon,zLon=rotatePoints(xLon,yLon,zLon,rotMatr)

    start=np.array([centX,centY,maxZ]).reshape((3,1))  # Start point of the sun vector    

    sunVector = np.array([np.sin(zenith) * np.cos(azimuth), np.sin(zenith) * np.sin(azimuth), np.cos(zenith)])
    sunVectorApparent = np.array([np.sin(zenith) * np.cos(azApparent), np.sin(zenith) * np.sin(azApparent), np.cos(zenith)])
    projPerp=np.array([0,sunVectorApparent[1],sunVectorApparent[2]])
    projPerp=projPerp/norm(projPerp)
    if sunVectorApparent[1]>=0.0:
        thetaPerp=  np.arccos(np.dot(projPerp, np.array([0, 0, 1])))
    else:
        thetaPerp= -np.arccos(np.dot(projPerp, np.array([0, 0, 1])))
    rotZ=rotationMatrixAroundZ(np.deg2rad(90.0+axRot_az))
    rotAxis=rotationMatrixAroundZ(np.deg2rad(axRot_az)) @ np.array([1,0,0])
    rotatedTrack=rotationMatrixAroundAxis(rotAxis,-thetaPerp)
  
    rr=np.matmul(rotatedTrack,rotZ)
    x,y,z=rotatePoints(x,y,z,rr)
    xAp,yAp,zAp=rotatePoints(xAp,yAp,zAp,rr)
    xLon,yLon,zLon=rotatePoints(xLon,yLon,zLon,rr)
    
    start=rr @ start
    end = start + sunVector.reshape((3,1)) * length

    normalVect=rr @ np.array([0,0,1])
    end2=start+normalVect.reshape((3,1))*length

    xArc,yArc, zArc=arcCircle3D(length,start,end,end2, nDiscr=25)

    incAngle=np.rad2deg(np.arccos(np.dot(sunVector,normalVect)/(norm(sunVector)*norm(normalVect))))

    coordSun=start+sunVector.reshape((3,1))*length
    xSun,ySun,zSun=sphereSurface(radius=0.5,coord=[coordSun[0], coordSun[1], coordSun[2]], nDiscr=8)

    if sunVector[1]>=0.0:
        tt=length/2.0
    else:
        tt=-length/2.0

    xHor,yHor,zHor=xHor+start[0],yHor+start[1]+tt,zHor+start[2]
    xLon,yLon,zLon=xLon+start[0],yLon+start[1],zLon+start[2]
    xAp,yAp,zAp=xAp+start[0],yAp+start[1],zAp+start[2]
    projSunHor=(np.array([sunVector[0],sunVector[1],0])).reshape(3,1)

    xArcZen,yArcZen, zArcZen=arcCircle3D(length/2.0,start,start+np.array([0,0,0.5*length]).reshape(3,1),end, nDiscr=25)
    xArcAz,yArcAz = arcCircle2DAngle(length/2.0,start,azimuth,nDiscr=50)
    zArcAz=np.zeros(len(xArcAz))+start[2]

    projSunHor=projSunHor/norm(projSunHor)
    projSunHor=projSunHor*length/2.0
    return

#endregion

indTest=16
initialCompute(lon, lat, year, month, day)
azimuth = np.deg2rad(azArray[indTest])
zenith = np.deg2rad(zenArray[indTest])

comp(np.deg2rad(zenArray[indTest]), np.deg2rad(azArray[indTest]),axRot_az)

#region Plotly section
# ============================================================================
#                               Draw figure section
# ============================================================================

def drawFig():
    maxX=np.max(x)
    fig = go.Figure()
    test=False
    fig.add_trace(go.Surface(x=x, y=y, z=z, colorscale=[[0, 'grey'], [1, 'grey']], opacity=0.9, showscale=False, name='Parabolic Cylinder'))
    color = 'orange'
    color2='blue'
    fig.add_trace(go.Cone(
        x=[start[0,0]], y=[start[1,0]], z=[start[2,0]],
        u=[-sunVector[0]], v=[-sunVector[1]], w=[-sunVector[2]],
        sizemode="absolute",
        sizeref=1.,
        colorscale=[[0, color], [1, color]],
        showscale=False,
        visible=False
    ))
    fig.add_trace(go.Scatter3d(
        x=[start[0,0], end[0,0]],
        y=[start[1,0], end[1,0]],
        z=[start[2,0], end[2,0]],
        mode='lines',
        line=dict(color=color, width=10),
        name='Sun Vector'
    ))
    fig.add_trace(go.Cone(
        x=[end2[0,0]], y=[end2[1,0]], z=[end2[2,0]],
        u=[normalVect[0]*length], v=[normalVect[1]*length], w=[normalVect[2]*length],
        sizemode="absolute",
        sizeref=1.,
        colorscale=[[0, color2], [1, color2]],
        showscale=False,
    ))
    fig.add_trace(go.Scatter3d(
        x=[start[0,0], end2[0,0]],
        y=[start[1,0], end2[1,0]],
        z=[start[2,0], end2[2,0]],
        mode='lines',
        line=dict(color=color2, width=10),
        name='Normal Unit Vector'
    ))

    fig.add_trace(go.Scatter3d(
        x=xArc,
        y=yArc,
        z=zArc,
        mode='lines',
        line=dict(color='black', width=5),
        name='IncAngle'
    ))

    fig.add_trace(go.Scatter3d(
        x=xArcZen,
        y=yArcZen,
        z=zArcZen,
        mode='lines',
        line=dict(color='black', width=5),
        name='ZenithAngle'
    ))


    fig.add_trace(go.Scatter3d(
        x=xArcAz,
        y=yArcAz,
        z=zArcAz,
        mode='lines',
        line=dict(color='red', width=5),
        name='AzimuthAngle'
    ))

    fig.add_trace(go.Surface(
        x=xAp, y=yAp, z=zAp,
        colorscale='gray',
        opacity=0.5,
        showscale=False,
        name='PlaneAperture'
    ))

    fig.add_trace(go.Surface(
        x=xLon, y=yLon, z=zLon,
        colorscale='blues',
        opacity=0.5,
        showscale=False,
        name='LongPlane'
    ))

    fig.add_trace(go.Surface(
        x=xSun, y=ySun, z=zSun,
        colorscale='solar',
        opacity=1.0,
        showscale=False,
        name='Sun'
    ))

    fig.add_trace(go.Scatter3d(
        x=endArr[0,iiSun:iiFin+1],
        y=endArr[1,iiSun:iiFin+1],
        z=endArr[2,iiSun:iiFin+1],
        mode='lines',
        line=dict(color='yellow', width=5),
        name='Sun Path'
    ))

    fig.add_trace(go.Cone(
        x=[start[0,0]], y=[start[1,0]], z=[start[2,0]+length],
        u=[0], v=[0], w=[length],
        sizemode="absolute",
        sizeref=1.,
        colorscale=[[0, 'black'], [1, color]],
        showscale=False
        ))

    fig.add_trace(go.Scatter3d(
        x=[start[0,0], start[0,0]],
        y=[start[1,0], start[1,0]],
        z=[start[2,0], start[2,0]+length],
        mode='lines',
        line=dict(color='black', width=10),
        name='VertDir'
    ))

    fig.add_trace(go.Cone(
        x=[start[0,0]+length], y=[start[1,0]], z=[start[2,0]],
        u=[length], v=[0], w=[0],
        sizemode="absolute",
        sizeref=1.,
        colorscale=[[0, 'black'], [1, color]],
        showscale=False
        ))

    fig.add_trace(go.Scatter3d(
        x=[start[0,0], start[0,0]+length],
        y=[start[1,0], start[1,0]],
        z=[start[2,0], start[2,0]],
        mode='lines',
        line=dict(color='black', width=10),
        name='HorzDir'
    ))

    fig.add_trace(go.Surface(
        x=xHor, y=yHor, z=zHor,
        colorscale='blues',
        opacity=0.5,
        showscale=False,
        name='Horizontal Plane'
    ))

    fig.add_trace(go.Scatter3d(
    x=[start[0,0], start[0,0]+projSunHor[0,0]],
    y=[start[1,0], start[1,0]+projSunHor[1,0]],
    z=[start[2,0], start[2,0]+projSunHor[2,0]],
    mode='lines',
    line=dict(color='black', width=2),
    name='HorzDir'
    ))

    fig.update_layout(
        scene=dict(
            annotations=[
            dict(
                showarrow=False,
                x=xArc[len(xArc)//2],
                y=yArc[len(xArc)//2],
                z=zArc[len(xArc)//2],
                text="θinc: "+str(format(incAngle, '.2f'))+"°",
                xanchor="left",
                xshift=10,
                opacity=0.7),
            dict(
                showarrow=False,
                x=start[0,0]+length,
                y=start[1,0],
                z=start[2,0],
                text="North",
                yshift=25,
                opacity=0.7,
                font=dict(
                    color="black",
                    size=16
                ),
                ),      
            dict(
                showarrow=False,
                x=xArcAz[len(xArcAz)//2],
                y=yArcAz[len(xArcAz)//2],
                z=zArcAz[len(xArcAz)//2],
                text="γ: "+str(format(az, '.2f'))+"°",
                yshift=25,
                textangle=0,
                font=dict(
                    color="black",
                    size=12
                ),
                ),
            dict(
                showarrow=True,
                x=xArcZen[len(xArcZen)//2],
                y=yArcZen[len(xArcZen)//2],
                z=zArcZen[len(xArcZen)//2],
                text="θzen: "+str(format(zen, '.2f'))+"°",
                xanchor="left",
                yanchor="bottom"
            )]
        ),
    )
    fig.update_layout(scene_aspectmode='manual',
        scene_aspectratio=dict(x=1, y=1, z=1))
    fig.update_layout(scene=dict(
        xaxis=dict(range=[-length*1.2, length*1.2]),
        yaxis=dict(range=[-length*1.2, length*1.2]),
        zaxis=dict(range=[-length/2, length*1.2]),
        ))
    fig.update_layout(
        margin=dict(l=20, r=20, t=20, b=20),
        showlegend=False,
        height=800  # Set the height of the figure
    )
    return fig
#endregion

# Update sun vector and related calculations
def update_figure(day, month, year,timeCurr, lat,lon, axRot_az,timeZone ):
    azimuth=np.zeros(1)
    zenith=np.zeros(1)

    year, month, day, timeCurr=timeZoneToUTC(year, month, day, timeCurr, timeZone)

    # Calculate hours and minutes from timeCurr (which is in minutes)
    
    # Convert year, month, day to strings with proper formatting
    year_str = str(year)
    month_str = f"{month:02d}"  # Ensure two digits with leading zero if needed
    day_str = f"{day:02d}"      # Ensure two digits with leading zero if needed
    
    hours = timeCurr // 60
    minutes = timeCurr % 60
    
    # Format the time part with leading zeros
    time_str = f"{hours:02d}:{minutes:02d}:00"
    
    # Create the complete datetime64 string
    stringDay = f"{year_str}-{month_str}-{day_str}"
    
    # For using in sunpos function
    lat_array = np.array([lat])
    lon_array = np.array([lon])
    
    # Create the datetime64 object
    currTime = np.datetime64(f"{stringDay}T{time_str}")
    
    # Get the sun position for the specific time
    a, b = sp.sunpos(currTime, lat_array, lon_array, 0)[:2]
    azimuth[0]=a[0]
    zenith[0]=b[0]   
    
    # Prepare for the hourly calculations
           
    #azArray[int(ii)],zenArray[int(ii)] = sp.sunpos(currTime,lat,lon,0)[:2] #discard RA, dec, H

    zenithIn=np.deg2rad(zenith[0])
    azimuthIn=np.deg2rad(azimuth[0])
    initialCompute(lon, lat, year, month, day)
    
    # Only run the 24-hour loop if date or location parameters have changed
    # This prevents unnecessary calculations when only time is changed
    global prev_day, prev_month, prev_year, prev_lat, prev_lon, prev_axRot_az
    
    # Initialize the tracking variables if they don't exist
    if 'prev_day' not in globals():
        prev_day, prev_month, prev_year = -1, -1, -1
        prev_lat, prev_lon, prev_axRot_az = -999, -999, -999
    
    # Check if date or location parameters have changed
    date_loc_changed = (
        day != prev_day or 
        month != prev_month or 
        year != prev_year or 
        lat != prev_lat or 
        lon != prev_lon or
        axRot_az != prev_axRot_az
    )
    
    if date_loc_changed:
        for ii in range(0,24):
            comp(np.deg2rad(zenArray[ii]), np.deg2rad(azArray[ii]), axRot_az)
            endArr[:,ii] = start[:,0] + sunVector * length
        
        # Update the previous parameters
        prev_day, prev_month, prev_year = day, month, year
        prev_lat, prev_lon, prev_axRot_az = lat, lon, axRot_az

    comp(zenithIn, azimuthIn, axRot_az)
    #initialCompute(lon, lat, year, month, day)
    #azimuth = np.deg2rad(azArray[hour])
    #zenith = np.deg2rad(zenArray[hour])

 
    with fig.batch_update():

        fig.update_layout(scene_aspectmode='manual',
                  scene_aspectratio=dict(x=1, y=1, z=1))
        fig.data[0].update(x=x, y=y, z=z)
        fig.data[1].update(x=[start[0,0]], y=[start[1,0]], z=[start[2,0]])
        fig.data[2].update(x=[start[0,0], end[0,0]], y=[start[1,0], end[1,0]], z=[start[2,0], end[2,0]])
        fig.data[3].update(x=[end2[0,0]], y=[end2[1,0]], z=[end2[2,0]],u=[normalVect[0]*length], v=[normalVect[1]*length], w=[normalVect[2]*length])
        fig.data[4].update(x=[start[0,0], end2[0,0]], y=[start[1,0], end2[1,0]], z=[start[2,0], end2[2,0]])
        fig.data[5].update(x=xArc, y=yArc, z=zArc)
        fig.data[6].update(x=xArcZen, y=yArcZen, z=zArcZen)
        fig.data[7].update(x=xArcAz, y=yArcAz, z=zArcAz)
        fig.data[8].update(x=xAp, y=yAp, z=zAp)
        fig.data[9].update(x=xLon, y=yLon, z=zLon)
        fig.data[10].update(x=xSun, y=ySun, z=zSun)
        fig.data[11].update(x=endArr[0,iiSun:iiFin+1], y=endArr[1,iiSun:iiFin+1], z=endArr[2,iiSun:iiFin+1])
        fig.data[12].update(x=[start[0,0]], y=[start[1,0]], z=[start[2,0]+length])
        fig.data[13].update(x=[start[0,0], start[0,0]], y=[start[1,0], start[1,0]], z=[start[2,0], start[2,0]+length])
        fig.data[14].update(x=[start[0,0]+length], y=[start[1,0]], z=[start[2,0]],u=[length], v=[0], w=[0])
        fig.data[15].update(x=[start[0,0], start[0,0]+length], y=[start[1,0], start[1,0]], z=[start[2,0], start[2,0]])
        fig.data[16].update(x=xHor, y=yHor, z=zHor)
        fig.data[17].update(x=[start[0,0], start[0,0]+projSunHor[0,0]], y=[start[1,0], start[1,0]+projSunHor[1,0]], z=[start[2,0], start[2,0]+projSunHor[2,0]])
        fig.update_layout(
            scene=dict(
                annotations=[
                dict(
                    showarrow=False,
                    x=xArc[len(xArc)//2],
                    y=yArc[len(xArc)//2],
                    z=zArc[len(xArc)//2],
                    text="θinc: "+str(format(incAngle, '.2f'))+"°",
                    xanchor="left",
                    xshift=10,
                    opacity=0.7),
                dict(
                    showarrow=False,
                    x=start[0,0]+length,
                    y=start[1,0],
                    z=start[2,0],
                    text="North",
                    yshift=25,
                    opacity=0.7,
                    font=dict(
                        color="black",
                        size=16
                    ),
                    ),      
                dict(
                    showarrow=False,
                    x=xArcAz[len(xArcAz)//2],
                    y=yArcAz[len(xArcAz)//2],
                    z=zArcAz[len(xArcAz)//2],
                    text="γ: "+str(format(az, '.2f'))+"°",
                    yshift=25,
                    textangle=0,
                    font=dict(
                        color="black",
                        size=12
                    ),
                    ),
                dict(
                    showarrow=True,
                    x=xArcZen[len(xArcZen)//2],
                    y=yArcZen[len(xArcZen)//2],
                    z=zArcZen[len(xArcZen)//2],
                    text="θzen: "+str(format(zen, '.2f'))+"°",
                    xanchor="left",
                    yanchor="bottom"
                )]
            ),
        )        
        fig.show()

# Helper function to format time for display

# Update day slider max value when month or year changes
def update_day_slider(*args):
    max_days = days_in_month(monthSlider.value, yearSlider.value)
    daySlider.max = max_days
    if daySlider.value > max_days:
        daySlider.value = max_days

#if firstRun==True:
fig=drawFig()
    #fig.update_layout(autosize=True) # remove height=800
   #region print results
    #print('Azimuth: ', format(np.rad2deg(azimuth), '.2f'), '°')  
    #print('Zenith: ', format(np.rad2deg(zenith), '.2f'), '°')
    #print('Incidence Angle: ', format(incAngle, '.2f'), '°')
    #print('thetaPerp: ', format(np.rad2deg(thetaPerp), '.2f'), '°')
#    firstRun=False

monthSlider=IntSlider(min=1, max=12, step=1, value=month, description="Month [m]", layout=Layout(width='100%'))
yearSlider=IntSlider(min=2000, max=2100, step=1, value=year, description="Year [y]", layout=Layout(width='100%'))
daySlider=IntSlider(min=1, max=days_in_month(int(month), int(year)), step=1, value=int(day), description="Day [d]", layout=Layout(width='100%'))
timeSlider = IntSlider(min=0, max=24*60-1, step=1, value=12*60, description="Time [12:00]", layout=Layout(width='100%'))

def update_time_description(change):
    hours = timeSlider.value // 60
    minutes = timeSlider.value % 60
    timeSlider.description = f"Time [{hours:02d}:{minutes:02d}]"

timeSlider.observe(update_time_description, names='value')
update_time_description(None)  # Initialize the description

# Register observers
monthSlider.observe(update_day_slider, names='value')
yearSlider.observe(update_day_slider, names='value')
latSlider=FloatSlider(min=-89, max=89, step=0.1, value=lat, description="Latitude [°]", layout=Layout(width='100%'))
lonSlider=FloatSlider(min=-180, max=180, step=0.1, value=lon, description="Longitude[°]", layout=Layout(width='100%'))
axRotAzSlider=FloatSlider(min=0, max=180, step=1, value=axRot_az, description="AxOrient [°]", layout=Layout(width='100%'))
timeZoneText=BoundedFloatText(value=0.0, min=-12.0, max=12.0, step=0.1, description='TimeZone [h]', disabled=False)

slider_box = HBox([
    VBox([yearSlider, monthSlider, daySlider, timeSlider], layout=Layout(width='50%')),
    VBox([latSlider, lonSlider, axRotAzSlider, timeZoneText], layout=Layout(width='50%'))
], layout=Layout(display='flex', flex_flow='row', align_items='center', width='100%'))
# Display the slider box
display(slider_box)

def update_time_description(change):
    hours = timeSlider.value // 60
    minutes = timeSlider.value % 60
    timeSlider.description = f"Time [{hours:02d}:{minutes:02d}]"

    timeSlider.observe(update_time_description, names='value')


# Connect the function to the sliders without creating duplicate UI elements
output = interactive_output(update_figure, {
    'day': daySlider,
    'month': monthSlider,
    'year': yearSlider,
    'timeCurr': timeSlider,
    'lat': latSlider,
    'lon': lonSlider,
    'axRot_az': axRotAzSlider,
    'timeZone': timeZoneText
})
display(output)


#endregion

# Matplotlib section
#fig2 = plt.figure()
#ax = fig2.add_subplot(111, projection='3d')
#ax.plot_surface(x, y, z, cmap='viridis')
#ax.set_xlabel('X')
#ax.set_ylabel('Y')
#ax.set_zlabel('Z')
#ax.quiver(end[0], end[1], end[2], -sunVector[0], -sunVector[1], -sunVector[2], length=length, color=color)

#ax.plot_surface(xAp, yAp, zAp, alpha=0.5, color='gray')
#plt.show()
#a=1





# %%



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\giogi\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\giogi\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\giogi\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\giogi\AppData\Local\Programs\Python\Python310\lib\site-packages\trait

AttributeError: _ARRAY_API not found

Output()